In [226]:
import pandas as pd
import os

In [227]:
sql_login_params = {"username": "wang2506_ro", "password": "fnVnwcCS7iT45EsA"}
sf_login_params = {
    "grant_type": "password",
    "client_id": "3MVG95jctIhbyCppj0SNJ75IsZ1y8UPGZtSNF4j8FNVXz.De8Lu4jHm3rjRosAtsHy6qjHx3i4S_QbQzvBePG",
    "client_secret": "D1623C6D3607D4FC8004B92C761DFB6C1F70CCD129C5501E357028DFA00F5764",
    "username":"wang2506@purdue.edu",
    "password":"mathisfun992728tXYwCb5yMm6XtohbTRw8ZdBbi",
}

## Load input data

Hubspot data: $ cwd+all_contacts.xlsx

In [244]:
#import data
cwd = os.getcwd()
print(cwd)
temp = pd.read_csv('all-contacts.csv',chunksize=1000)

C:\Users\henry\Desktop\PhD Research\Nanohub RA\nanohub_salesforce_integ-master\hubspot_table_generation


In [245]:
# filtering the full .csv file for only 
# firstname, lastname, champion status, Number of times contacted, Number of Sales Activities
# lifecycle stage, last contacted?
t_list = []
for chunk in temp:
    t_df = pd.DataFrame() #['First Name','Last Name','Champion','Disciplines','LinkedIn Bio','Venue','Contact Type']
    t_df['FirstName'] = chunk.loc[:,'First Name']
    t_df['LastName'] = chunk.loc[:,'Last Name']
    t_df['Champion'] = chunk.loc[:,'Champion']
    t_df['Linkedin_Bio'] = chunk.loc[:,'LinkedIn Bio']
    t_df['Venue'] = chunk.loc[:,'Venue']
    t_df['Contact_Type'] = chunk.loc[:,'Contact Type']
    t_df['nanoHUB_user_ID'] = chunk.loc[:,'nanoHUB ID']
    t_df['Email'] = chunk.loc[:,'Email']
    
    t_discplines = chunk.loc[:,'DISCIPLINES']
    t_field_of_study = chunk.loc[:,'Field of study']
    t_df['Disciplines'] = t_discplines+t_field_of_study
    
    t_list.append(t_df)

In [246]:
df_filter = pd.concat(t_list)
df_filter = df_filter.dropna(subset=['FirstName','LastName'])
df_filter = df_filter.reset_index(drop=True)
display(df_filter.head(2))

,FirstName,LastName,Champion,Linkedin_Bio,Venue,Contact_Type,nanoHUB_user_ID,Email,Disciplines
0,ST,H Tawfick,NaN,NaN,NaN,nanoMFG,186599.0,tawfick@illinois.edu,NaN
1,Jay,R Roloff,NaN,NaN,NaN,nanoMFG,199490.0,jayr@illinois.edu,NaN
2,Elif,Ertekin,NaN,NaN,MRS Fall 2019,nanoMFG,171646.0,ertekin@illinois.edu,NaN
3,Travis,Merrick,NaN,NaN,NaN,Purdue NCN Associate; nanoBIO,206944.0,tmerric@purdue.edu,NaN
4,Paul,Macklin,NaN,NaN,NaN,nanoBIO,178498.0,macklinp@iu.edu,NaN
...,...,...,...,...,...,...,...,...,...
185000,Southwest,Center for Microsystems Education (SCME),NaN,Reasearch Professor at University of New Mexico,NaN,NaN,144398.0,mpleil@unm.edu,NaN
185001,MJ,Willis,NaN,NaN,NaN,NaN,162177.0,mjwillis@comcast.net,NaN
185002,Barbara,Lopez,NaN,NaN,NaN,NaN,147235.0,botero@unm.edu,NaN
185003,Michael,Zentner,NaN,Senior Research Scientist at Purdue University,NaN,NaN,NaN,mzentner@purdue.edu,NaN


In [247]:
# split last name into middle and last name
def split_name(this_name):
    name_list = this_name.split(' ')
    if len(name_list) == 1:
        return pd.Series([name_list[0]])
    else:
        return pd.Series([name_list[-1]])

df_filter['LastName'] = df_filter['LastName'].apply(split_name)

,0
0,Tawfick
1,Roloff
2,Ertekin
3,Merrick
4,Macklin
...,...
185000,(SCME)
185001,Willis
185002,Lopez
185003,Zentner


In [248]:
# adjusting Champion - add HS in front
def hs_add(champ):
    if champ == 'Yes':
        return pd.Series(['HS champion'])
#df_filter['Champion'].apply(lambda x: 'HS champion' if x == 'Yes' else None)
#df_filter['Champion'].apply(hs_add)
#df_filter['Champion'][53] = 'HS champion'
    
for i,j in enumerate(df_filter['Champion']):
    if j == 'Yes':
        df_filter['Champion'][i] = 'HS champion'

C:\Users\henry\Anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [249]:
# adjusting Venue column - add HS in front
for i,j in enumerate(df_filter['Venue']):
    if type(j) == str:
        df_filter['Venue'][i] = 'HS '+ df_filter['Venue'][i]
df_filter['Venue'].head(5)

C:\Users\henry\Anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


0                 NaN
1                 NaN
2    HS MRS Fall 2019
3                 NaN
4                 NaN
Name: Venue, dtype: object

## split dataframe into two separate dataframes - one for leads, other for contacts

In [250]:
# Obtain contacts metadata from Salesforce
from DB2SalesforceAPI import DB2SalesforceAPI

# create DB2 to Salesforce API object
db_s = DB2SalesforceAPI(sf_login_params)

# specify Salesforce object ID and external ID
contact_metadata = db_s.get_obj_metadata('Contact')

Obtained Salesforce access token ...... True


In [251]:
import json

# get all fields
field_list = json.loads(contact_metadata)['fields']

# list of available field API names
available_fields = list()

for this_field in field_list:
    this_name = this_field['name']
    
    if len(this_name) > 3:
        if this_name[-3:] == '__c':
            # this is a custom-made field
            available_fields.append(this_name[:-3])
            
# display
display(available_fields[:5])

['Type',
 'nanoHUB_user_ID',
 'nanoHUB_username',
 'Organization_Type',
 'nanoHUB_registration_date']

In [269]:
#basically, if the name in df_filter has a non-NaN/Nonetype entry for nanoHUB_user_ID column
#then they already exist as a contact
ci1 = df_filter['nanoHUB_user_ID'].notna()
ci2 = df_filter['Champion'].notna()
contacts_indices = ci1|ci2

import numpy as np
leads_indices = np.ones(contacts_indices.shape[0],dtype=bool) ^ contacts_indices
#leads_indices = df_filter['nanoHUB_user_ID'].isna() & df_filter['Champion'].isna()

## Match data with Salesforce format [CONTACTS]

In [278]:
# get contacts specific dataframe
contacts_df = df_filter[contacts_indices]
# find columns that have matched fields in Salesforce
matched_columns = set(contacts_df.columns).intersection(set(available_fields))

# display
display(matched_columns)

{'Champion',
 'Contact_Type',
 'Disciplines',
 'Linkedin_Bio',
 'Venue',
 'nanoHUB_user_ID'}

In [279]:
df_sf_contacts = pd.DataFrame()

# names, nanohub id, and email
df_sf_contacts['firstname'] = contacts_df['FirstName']
df_sf_contacts['lastname'] = contacts_df['LastName']
df_sf_contacts['nanoHUB_user_ID__c']            = contacts_df['nanoHUB_user_ID']
df_sf_contacts['Email']                         = contacts_df['Email'].fillna('').apply(lambda x: '' if '@invalid' in x else x)

# hubspot specific qualities
df_sf_contacts['Champion__c'] = contacts_df['Champion']
df_sf_contacts['Contact_Type__c'] = contacts_df['Contact_Type']
df_sf_contacts['Venue__c'] = contacts_df['Venue']
df_sf_contacts['Linkedin_Bio__c'] = contacts_df['Linkedin_Bio']
df_sf_contacts['Disciplines__c'] = contacts_df['Disciplines']


# display
df_sf_contacts.head(2).T

,0,1
firstname,ST,Jay
lastname,H Tawfick,R Roloff
nanoHUB_user_ID__c,186599,199490
Email,tawfick@illinois.edu,jayr@illinois.edu
Champion__c,NaN,NaN
Contact_Type__c,nanoMFG,nanoMFG
Venue__c,NaN,NaN
Linkedin_Bio__c,NaN,NaN
Disciplines__c,NaN,NaN


In [280]:
external_id = 'Email'
object_id = 'Contact'

# create DB2 to Salesforce API object
db_s = DB2SalesforceAPI(sf_login_params)

# specify Salesforce object ID and external ID
db_s.object_id = object_id 
db_s.external_id = external_id

Obtained Salesforce access token ...... True


In [281]:
db_s.send_data(df_sf_contacts)

[Success] Bulk job creation successful. Job ID = 7505w00000O3dRZAAZ
hello
[Success] CSV upload successful. Job ID = 7505w00000O3dRZAAZ
[Success] Closing job successful. Job ID = 7505w00000O3dRZAAZ


In [282]:
# check status
db_s.check_bulk_status()

{'id': '7505w00000O3dRZAAZ',
 'operation': 'upsert',
 'object': 'Contact',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2020-07-15T18:05:47.000+0000',
 'systemModstamp': '2020-07-15T18:06:02.000+0000',
 'state': 'UploadComplete',
 'externalIdFieldName': 'Email',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 0,
 'numberRecordsFailed': 0,
 'retries': 0,
 'totalProcessingTime': 0,
 'apiActiveProcessingTime': 0,
 'apexProcessingTime': 0}

In [283]:
# check status
from pprint import pprint

pprint(db_s.check_bulk_failed_results())

''


## Match data with Salesforce format [LEADS]

In [284]:
# get leads specific dataframe
leads_df = df_filter[leads_indices]
# find columns that have matched fields in Salesforce
matched_columns = set(leads_df.columns).intersection(set(available_fields))

# display
display(matched_columns)

{'Champion',
 'Contact_Type',
 'Disciplines',
 'Linkedin_Bio',
 'Venue',
 'nanoHUB_user_ID'}

In [296]:
df_sf_leads = pd.DataFrame()

# create a new column for object name
df_sf_leads['firstname'] = leads_df['FirstName']
df_sf_leads['lastname'] = leads_df['LastName']
df_sf_leads['Venue__c'] = leads_df['Venue']
df_sf_leads['Email'] = leads_df['Email'].fillna('').apply(lambda x: '' if '@invalid' in x else x)
df_sf_leads['Contact_Type__c'] = leads_df['Contact_Type']
df_sf_leads['Linkedin_Bio__c'] = leads_df['Linkedin_Bio']
df_sf_leads['LeadSource'] = 'Hubspot'

df_sf_leads['Company'] = np.ones(leads_df['FirstName'].shape[0])*np.nan

# Company cannot be empty
df_sf_leads['Company'] = df_sf_leads['Company'].fillna('-')
df_sf_leads['Company'] = df_sf_leads['Company'].apply(lambda x: x if len(x.strip())>0 else '-')


# display
df_sf_leads.head(2).T

,7,8
firstname,Geoffrey,Hai
lastname,Fox,Pham
Venue__c,NaN,HS SACNAS 2019
Email,gcfexchange@gmail.com,haipham5050@gmail.com
Contact_Type__c,nanoBIO,URE Interest
Linkedin_Bio__c,NaN,NaN
LeadSource,Hubspot,Hubspot
Company,-,-


In [297]:
external_id = 'Email'
object_id = 'Lead'
# create DB2 to Salesforce API object
db_s = DB2SalesforceAPI(sf_login_params)

# specify Salesforce object ID and external ID
db_s.object_id = object_id 
db_s.external_id = external_id

Obtained Salesforce access token ...... True


In [298]:
db_s.send_data(df_sf_leads)

[Success] Bulk job creation successful. Job ID = 7505w00000O3dizAAB
hello
[Success] CSV upload successful. Job ID = 7505w00000O3dizAAB
[Success] Closing job successful. Job ID = 7505w00000O3dizAAB


In [299]:
# check status
db_s.check_bulk_status()

{'id': '7505w00000O3dizAAB',
 'operation': 'upsert',
 'object': 'Lead',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2020-07-15T18:23:07.000+0000',
 'systemModstamp': '2020-07-15T18:23:08.000+0000',
 'state': 'UploadComplete',
 'externalIdFieldName': 'Email',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 0,
 'numberRecordsFailed': 0,
 'retries': 0,
 'totalProcessingTime': 0,
 'apiActiveProcessingTime': 0,
 'apexProcessingTime': 0}

In [294]:
# check status
from pprint import pprint

pprint(db_s.check_bulk_failed_results())

''
